# Using PyMC3

PyMC3 is a Python package for doing MCMC using a variety of samplers, including Metropolis, Slice and Hamiltonian Monte Carlo. See [Probabilistic Programming in Python using PyMC](http://arxiv.org/abs/1507.08050) for a description. The GitHub [site](https://github.com/pymc-devs/pymc3) also has many examples and links for further exploration.

Note: [PyMC4](https://github.com/pymc-devs/pymc4) is based on TensorFlow rather than Theano but will have a similar API so everyghitn learned should be transferable.

```bash
! pip install --quiet arviz
! pip install --quiet pymc3
! pip install --quiet daft
! pip install --quiet seaborn
```

```bash
! conda install --yes --quiet mkl-service
```

In [ ]:
import warnings

warnings.simplefilter('ignore', UserWarning)

**Other resources**

Some examples are adapted from:

- [Probabilistic Programming & Bayesian Methods for Hackers](http://camdavidsonpilon.github.io/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/)
- [MCMC tutorial series](https://theclevermachine.wordpress.com/2012/11/19/a-gentle-introduction-to-markov-chain-monte-carlo-mcmc/)

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import numpy.random as rng
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc3 as pm
import scipy.stats as stats
import daft
import arviz as az

In [ ]:
import theano
theano.config.warn.round=False

In [ ]:
sns.set_context('notebook')
plt.style.use('seaborn-darkgrid')

## Introduction to PyMC3

### Distributions in pymc3

In [ ]:
print('\n'.join([d for d in dir(pm.distributions) if d[0].isupper()]))

In [ ]:
d = pm.Normal.dist(mu=0, sd=1)

In [ ]:
d.dist()

Random samples

In [ ]:
d.random(size=5)

Log probability

In [ ]:
d.logp(0).eval()

#### Custom distributions

The pymc3 algorithms generally only work with the log probability of a distribution. Hence it is easy to define custom distributions to use in your models by providing a `logp` function.

In [ ]:
def logp(x, μ=0, σ=1):
    """Normal distribtuion."""
    return -0.5*np.log(2*np.pi) - np.log(σ) - (x-μ)**2/(2*σ**2)

In [ ]:
d = pm.DensityDist.dist(logp)

In [ ]:
d.logp(0)

## Example: Estimating coin bias

We start with the simplest model - that of determining the bias of a coin from observed outcomes.

Here the prior is a beta distribution with paramters $a$ and $b$, the likelihood assumes that the number of heads follows a binomial distribution with parameters $n$ and $p$, and we wish to estimate the posterior distribution of $\theta = p$.

### Prior = Beta distribution

In [ ]:
pars = [(0.5, 0.5), (1,1), (1,10), (5,5), (10, 1), (10,10)]

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
for i, (a, b) in enumerate(pars):
    ax = axes[i // 3, i % 3]
    θ = np.linspace(0, 1, 100)
    dens = stats.beta(a, b).pdf(θ)
    ax.plot(θ, dens)
    ax.set_title('a=%.1f, b=%.1f' % (a, b))

### Posterior = bionmial distribution

In [ ]:
pars = [0, 0.2, 0.4, 0.6, 0.8, 1]
n = 10

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
for i, p in enumerate(pars):
    ax = axes[i // 3, i % 3]
    θ = np.arange(0, n+1)
    dens = stats.binom(n, p).pmf(θ)
    ax.stem(θ, dens)
    ax.set_title('n=%.1f, p=%.1f' % (n, p))

### Setting up the model    

In [ ]:
n = 100
heads = 61

#### Analytical solution

In [ ]:
a, b = 10, 10
prior = stats.beta(a, b)
post = stats.beta(heads+a, n-heads+b)
ci = post.interval(0.95)

xs = np.linspace(0, 1, 100)
plt.plot(prior.pdf(xs), label='Prior')
plt.plot(post.pdf(xs), c='green', label='Posterior')
plt.axvline(100*heads/n, c='red', alpha=0.4, label='MLE')
plt.xlim([0, 100])
plt.axhline(0.3, ci[0], ci[1], c='black', linewidth=2, label='95% CI');
plt.legend()
pass

#### Graphical model

In [ ]:
pgm = daft.PGM(shape=[2.5, 3.0], origin=[0, -0.5])

pgm.add_node(daft.Node("alpha", r"$\alpha$", 0.5, 2, fixed=True))
pgm.add_node(daft.Node("beta", r"$\beta$", 1.5, 2, fixed=True))
pgm.add_node(daft.Node("p", r"$p$", 1, 1))
pgm.add_node(daft.Node("n", r"$n$", 2, 0, fixed=True))
pgm.add_node(daft.Node("y", r"$y$", 1, 0, observed=True))

pgm.add_edge("alpha", "p")
pgm.add_edge("beta", "p")
pgm.add_edge("n", "y")
pgm.add_edge("p", "y")

pgm.render()
pass

### The Model context

When you specify a model, you are adding nodes to a computation graph. When executed, the graph is compiled via Theno. Hence, `pymc3` uses the Model context manager to automatically add new nodes.

In [ ]:
niter = 2000
with pm.Model() as coin_context:
    p = pm.Beta('p', alpha=2, beta=2)
    y = pm.Binomial('y', n=n, p=p, observed=heads)
    trace = pm.sample(niter)

In [ ]:
coin_context

#### Transformed prior variables

In [ ]:
coin_context.free_RVs

#### Prior variables

In [ ]:
coin_context.deterministics

#### Variables in likelihood

In [ ]:
coin_context.observed_RVs

### Under the hood

### Theano

Theano builds functions as mathematical expression graphs and compiles them into C for actual computation, making use of GPU resources where available.

Performing calculations in Theano generally follows the following 3 steps (from official docs):

- declare variables (a,b) and give their types
- build expressions for how to put those variables together
- compile expression graphs to functions in order to use them for computation.

In [ ]:
import theano
import theano.tensor as tt
theano.config.compute_test_value = "off"

This part builds symbolic expressions.

In [ ]:
a = tt.iscalar('a')
b = tt.iscalar('x')
c = a + b

This step compiles a function whose inputs are [a, b] and outputs are [c].

In [ ]:
f = theano.function([a, b], [c])

In [ ]:
f

In [ ]:
f(3, 4)

Within a model context, 

- when you add an unbounded variable, it is defined as a Theano variable and added to the prior part of the log posterior function
- when you add a bounded variable, a transformed version is defined as a Theano variable and and added to the log posterior function
    - The inverse transformation is used to define the original variable - this is a deterministic variable
- when you add an observed variable bound to data, the data is added to the likelihood part of the log posterior

See [PyMC3 and Theano](https://docs.pymc.io/PyMC3_and_Theano.html) for details.

In [ ]:
help(pm.sample)

### Specifying sampler (step) and multiple chains

In [ ]:
with coin_context:
    step = pm.Metropolis()
    t = pm.sample(niter, step=step, chains=8, random_seed=123)

### Samplers available

For continuous distributions, it is hard to beat NUTS and hence this is the default. To learn more, see [A Conceptual Introduction to Hamiltonian Monte Carlo](https://arxiv.org/pdf/1701.02434.pdf).

In [ ]:
print('\n'.join(m for m in dir(pm.step_methods) if m[0].isupper()))

Generally, the sampler will be automatically selected based on the type of the variable (discrete or continuous), but there are many samples that you can explicitly specify if you want to learn more about them or understand why an alternative would be better than the default for your problem.

In [ ]:
niter = 2000
with pm.Model() as normal_context:
    mu = pm.Normal('mu', mu=0, sd=100)
    sd = pm.HalfCauchy('sd', beta=2)
    y = pm.Normal('y', mu=mu, sd=sd, observed=xs)
    
    step1 = pm.Slice(vars=mu)
    step2 = pm.Metropolis(vars=sd)
    
    t = pm.sample(niter, step=[step1, step2])

In [ ]:
pm.traceplot(t)
pass

#### MAP estimate

In [ ]:
with pm.Model() as m:
    p = pm.Beta('p', alpha=2, beta=2)
    y = pm.Binomial('y', n=n, p=p, observed=heads)
    θ = pm.find_MAP()

In [ ]:
θ

#### Getting values from the trace

All the information about the posterior is in the trace, and it also provides statistics about the sampler.

In [ ]:
help(trace)

In [ ]:
trace.stat_names

In [ ]:
sns.distplot(trace.get_sampler_stats('model_logp'))
pass

In [ ]:
p = trace.get_values('p')
p.shape

In [ ]:
trace['p'].shape

#### Convert to `pandas` data frame for downstream processing

In [ ]:
df = pm.trace_to_dataframe(trace)
df.head()

#### Posterior distribution

In [ ]:
sns.distplot(trace['p'])
pass

#### Autocorrelation plot

In [ ]:
pm.autocorrplot(trace, varnames=['p'])
pass

#### Calculate effective sample size

$$
\hat{n}_{eff} = \frac{mn}{1 + 2 \sum_{t=1}^T \hat{\rho}_t}
$$

where $m$ is the number of chains, $n$ the number of steps per chain, $T$ the time when the autocorrelation first becomes negative, and $\hat{\rho}_t$ the autocorrelation at lag $t$.

In [ ]:
pm.effective_n(trace)

## Evaluate convergence

[Model checking and diagnostics](https://pymc-devs.github.io/pymc/modelchecking.html)

##### Gelman-Rubin

$$
\hat{R} = \sqrt{\frac{\hat{\text{Var}}(\theta | y)}{W}}
$$

where $W$ is the within-chain variance and the numeratro is the posterior variance estimate for the pooled traces.  Values greater than one indicate that one or more chains have not yet converged.

In [ ]:
pm.gelman_rubin(trace)

##### Geweke

Compares mean and variance of initial with later segments of a trace for a parameter. Should have absolute value less than 1 at convergence.

$$
z = \frac{\bar{\theta}_a - \bar{\theta}_b}{\sqrt{Var(\theta_a) + Var(\theta_b)}}
$$

In [ ]:
plt.plot(pm.geweke(trace['p'])[:,1], 'o')
plt.axhline(1, c='red')
plt.axhline(-1, c='red')
plt.gca().margins(0.05)
pass

#### Textual summary

In [ ]:
pm.summary(trace, varnames=['p'])

#### Visual summary

In [ ]:
pm.traceplot(trace, varnames=['p'])
pass

In [ ]:
pm.forestplot(trace)
pass

In [ ]:
pm.plot_posterior(trace)
pass

#### Prior predictive samples

In [ ]:
with coin_context:
    ps = pm.sample_prior_predictive(samples=1000)

In [ ]:
sns.distplot(ps['y'])
plt.axvline(heads, c='red')
pass

#### Posterior predictive samples

In [ ]:
with coin_context:
    ps = pm.sample_posterior_predictive(trace, samples=1000)

In [ ]:
sns.distplot(ps['y'])
plt.axvline(heads, c='red')
pass

## Saving traces

In [ ]:
pm.save_trace(trace, 'my_trace', overwrite=True)

You need to re-initialize the model when reloading.

In [ ]:
with pm.Model() as my_trace:
    p = pm.Beta('p', alpha=2, beta=2)
    y = pm.Binomial('y', n=n, p=p, observed=heads)
    tr = pm.load_trace('my_trace')

In [ ]:
pm.summary(tr)

It is probably a good practice to make model reuse convenient

In [ ]:
def build_model():
    with pm.Model() as m:
        p = pm.Beta('p', alpha=2, beta=2)
        y = pm.Binomial('y', n=n, p=p, observed=heads)
    return m

In [ ]:
m = build_model()

In [ ]:
with m:
    tr1 = pm.load_trace('my_trace')

In [ ]:
pm.summary(tr1)

## Estimating parameters of a normal distribution

In [ ]:
xs = np.random.normal(5, 2, 20)

## Sampling from prior

Just omit the `observed=` argument.

In [ ]:
with pm.Model() as prior_context:
    sigma = pm.Gamma('sigma', alpha=2.0, beta=1.0)
    mu = pm.Normal('mu', mu=0, sd=sigma)
    trace = pm.sample(niter, step=pm.Metropolis())

In [ ]:
pm.traceplot(trace, varnames=['mu', 'sigma'])
pass

## Sampling from posterior

In [ ]:
niter = 2000
with pm.Model() as normal_context:
    mu = pm.Normal('mu', mu=0, sd=100)
    sd = pm.HalfCauchy('sd', beta=2)
    y = pm.Normal('y', mu=mu, sd=sd, observed=xs)
    trace = pm.sample(niter)

#### Find Highest Posterior Density (Credible intervals)

In [ ]:
trace.varnames

In [ ]:
hpd = pm.hpd(trace['mu'])
hpd

In [ ]:
ax = pm.traceplot(trace, varnames=['mu'],)

ymin, ymax = ax[0,0].get_ylim()
y = ymin + 0.05*(ymax-ymin)
ax[0, 0].plot(hpd, [y,y], c='red')
pass

## Evaluating goodness-of-fit

WAIC is an approximation to the out-of-sample error and can be used for model comparison. Likelihood is dependent on model complexity and should not be used for model comparisons.

#### Cross-validation

In [ ]:
with normal_context:
    print(pm.loo(trace))

#### WAIC

In [ ]:
with normal_context:
    print(pm.waic(trace))

### Comparing models

Use precomputed models for convenience.

In [ ]:
data1 = az.load_arviz_data("non_centered_eight")
data2 = az.load_arviz_data("centered_eight")
compare_dict = {"non centered": data1, "centered": data2}
az.compare(compare_dict)

In [ ]:
az.compare(compare_dict, ic='LOO')

## Using a custom likelihood

In [ ]:
def logp(x, μ=0, σ=1):
    """Normal distribtuion."""
    return -0.5*np.log(2*np.pi) - np.log(σ) - (x-μ)**2/(2*σ**2)

In [ ]:
with pm.Model() as prior_context:
    mu = pm.Normal('mu', mu=0, sd=100)
    sd = pm.HalfCauchy('sd', beta=2)
    y = pm.DensityDist('y', logp, observed=dict(x=xs, μ=mu, σ=sd))
    custom_trace = pm.sample(niter)

In [ ]:
pm.trace_to_dataframe(custom_trace).mean()

### Variational methods available

To use a variational method, use `pm.fit` instead of `pm.sample`. We'll see examples of usage in another notebook.

In [ ]:
print('\n'.join(m for m in dir(pm.variational) if m[0].isupper()))